In [1]:
!pip install kaleido
!pip install skforecast
!pip install ordpy
!pip install EntropyHub

import os
import pandas as pd
import numpy as np
from skforecast.model_selection import backtesting_forecaster
from sklearn.linear_model import LinearRegression
from skforecast.ForecasterAutoreg import ForecasterAutoreg
import ordpy
import plotly.graph_objects as go
import EntropyHub as EH
import tqdm as tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.5/670.5 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 188.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.5 MB/s eta 0:00:00


In [2]:
import os
import pandas as pd

import statsmodels.api as sm

import kaleido

import plotly.graph_objects as go
import plotly.io as pio
import plotly.offline as poff
from plotly.subplots import make_subplots

import matplotlib.pyplot as plt

In [3]:
# hay que conectarse al drive en cada sesión
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
RUTA_DATA = "/content/drive/MyDrive/Proyecto_UTN_FRBA_Becarios/Códigos/2024/01 - Pronostico de demanda (experimento 1)/Datasets/"

data = pd.read_csv(os.path.join(RUTA_DATA, "dataset.csv"), index_col='fecha', parse_dates=['fecha'])
data = data.resample(rule='H', closed='left', label ='right').mean()
data.temp = data.temp.ffill()
data.tail()

,demanda,no_habil,temp,es_dia_calido,es_dia_frio
fecha,,,,,
2024-01-31 21:00:00,23976.806,0.0,31.80,1.0,0.0
2024-01-31 22:00:00,22933.568,0.0,30.85,1.0,0.0
2024-01-31 23:00:00,23470.519,0.0,30.25,1.0,0.0
2024-02-01 00:00:00,24386.953,0.0,29.10,0.5,0.0
2024-02-01 01:00:00,24334.169,0.0,28.70,0.5,0.0


# Fig - Entrenamiento y validacion


In [6]:
# se definen los intervalos de entranamiento y pronostico
end_train = '2023-12-31 23:59:00'
data_train = data.loc[:end_train, :].copy()
data_test = data.loc[end_train:, :].copy()

print(f"Train dates      : {data_train.index.min()} --- {data_train.index.max()}  (n={len(data_train)})")
print(f"Test dates       : {data_test.index.min()} --- {data_test.index.max()}  (n={len(data_test)})")

Train dates      : 2021-01-01 03:00:00 --- 2023-12-31 23:00:00  (n=26277)
Test dates       : 2024-01-01 00:00:00 --- 2024-02-01 01:00:00  (n=746)


In [16]:
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=data_train.index, y=data_train['demanda']/1000, name='Demanda', line=dict(color='black')))


temp = go.Scatter(x=data_train.index, y=data_train['temp'], name='Temperatura', line=dict(color='red'), opacity=0.5)
fig.add_trace(temp, secondary_y=True)

fig.update_layout(
    width=900, height=500,
    xaxis_title="Fecha",
    yaxis_title="Demanda [MW]",
    template="simple_white",
)

#fig.write_image("datos_entrenamiento.png",  engine="kaleido", scale=5)
fig.show()


In [ ]:
# prompt: muestra la temperatura con un alpha de 0.5

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=data_train.index, y=data_train['demanda']/1000, name='Demanda', line=dict(color='black')))


temp = go.Scatter(x=data_train.index, y=data_train['temp'], name='Temperatura', line=dict(color='red'), opacity=0.5)
fig.add_trace(temp, secondary_y=True)

fig.update_layout(
    width=900, height=500,
    xaxis_title="Fecha",
    yaxis_title="Demanda [MW]",
    template="simple_white",
)

#fig.write_image("datos_entrenamiento.png",  engine="kaleido", scale=5)
fig.show()


In [21]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=data_train.index, y=data_train['demanda']/1000, name='Registros', line=dict(color='black')))
#fig.add_trace(go.Scatter(x=data_test.index, y=data_test['demanda']/1000, name='Datos Futuros', line=dict(color='orange')))
fig.update_layout(
    width=600, height=400,
    xaxis_title="Fecha",
    yaxis_title="Demanda [MW]",
    template="simple_white",
)

#fig.write_image("datos_entrenamiento.png",  engine="kaleido", scale=5)
fig.show()

# Fig - Autocorrelacion

In [ ]:
import plotly.graph_objects as go

acf = sm.tsa.acf(data["demanda"], nlags=60)

fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(61)), y=acf, name='Autocorrelación', line=dict(color='black')))
fig.update_layout(
    width=600, height=400,
    xaxis_title="Horas",
    yaxis_title="Autocorrelación",
    template="simple_white",
    xaxis = dict(
        tickmode = 'linear',
        dtick = 6,
        showgrid=True,
    ),
    yaxis = dict(
        showgrid=True
    )
)

fig.write_image("autocorrelacion.png",  engine="kaleido", scale=5)
fig.show()

# Fig - detección colapso SADI

In [ ]:
# Calculo la entropía
entropia_ponderada = [0] * 24*7
for i in range(24*7, len(data.demanda), 1):
    h = ordpy.weighted_permutation_entropy(data.demanda[i-24*7:i], dx=3)
    entropia_ponderada.append(h)
data['entropia_ponderada'] = entropia_ponderada

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
trace1 = go.Scatter(x=data.index, y=data['demanda']/1000, name="Demanda", mode="lines", line=dict(color='black'))
trace2 = go.Scatter(x=data.index, y=data['entropia_ponderada'].iloc[24*7:-1], name="Entropía Ponderada", mode="lines", line=dict(color='red'))
fig.add_trace(trace1)
fig.add_trace(trace2, secondary_y=True)
fig.update_layout(
    xaxis_title="Años",
    yaxis_title="Demanda [MW]",
    width=600, height=400,
    template="simple_white",
)

fig.add_annotation(x="2023-2-22", y=21.4,
            text="Anomalía",
            showarrow=True,
            arrowcolor="yellow",
            bgcolor="yellow",
            bordercolor="black",
            arrowhead=1)


fig.update_yaxes(title_text="Entropía", secondary_y=True, range=[0, 1.0], dtick=0.2)
fig.write_image("entropia_ponderada.png",  engine="kaleido", scale=5)
fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
trace1 = go.Scatter(x=data.loc["2023-2-28":"2023-3-2"].index, y=data['demanda'].loc["2023-2-28":"2023-3-2"]/1000, name="Demanda", mode="lines", line=dict(color='black'))
trace2 = go.Scatter(x=data.loc["2023-2-28":"2023-3-2"].index, y=data['entropia_ponderada'].loc["2023-2-28":"2023-3-2"], name="Entropía Ponderada", mode="lines", line=dict(color='red'))
fig.add_trace(trace1)
fig.add_trace(trace2, secondary_y=True)
fig.update_layout(
    yaxis_title="Demanda [MW]",
    width=600, height=400,
    template="simple_white",

)
fig.update_yaxes(title_text="Entropía", secondary_y=True, range=[0.45, 0.7], dtick=0.1, title="Entropía")
#fig.update_xaxes(tickvals=[""])#showticklabels=False)
fig.write_image("entropia_ponderada_detalle.png",  engine="kaleido", scale=5)
fig.show()

# Fig - demanda semanal y entropia

In [ ]:
# generamos los pesos de entropia de acuerdo al avance durante el dia
entropia = [0] * 24
complejidad = [0] * 24
for i in range(24, len(data.demanda), 1):
    h, c = ordpy.complexity_entropy(data.demanda[i-24:i], dx=3)
    entropia.append(h)
    complejidad.append(c)
data['entropia'] = entropia
data['complejidad'] = complejidad



In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=data.loc["2022-07-10":"2022-09-11"].index, y=data['demanda'].loc["2022-07-10":"2022-09-11"]/1000, name='Demanda', line=dict(color='black')))
fig.add_trace(go.Scatter(x=data.loc["2022-07-10":"2022-09-11"].index, y=data['entropia'].loc["2022-07-10":"2022-09-11"], name='Entropía', line=dict(color='red')),  secondary_y=True)
fig.update_layout(
    width=600, height=400,
    xaxis_title="Fecha",
    yaxis_title="Demanda [MW]",
    template="simple_white",
)
fig.update_yaxes(range=[10, 35], secondary_y=False)
fig.update_yaxes(range=[0, 1], secondary_y=True, title="Entropía")


fig.add_shape(
    type="line",
    line_color="rgb(0, 5, 10)",
    x0="2022-07-31",
    y0=31,
    x1="2022-08-07",
    y1=31,
    line_width=4,
    label=dict(text="7 días", textposition="start"),
)

fig.write_image("entropia_semanal.png",  engine="kaleido", scale=5)
fig.show()

# Fig - Predictor ingenuo

In [ ]:
from skforecast.ForecasterBaseline import ForecasterEquivalentDate
from skforecast.model_selection import backtesting_forecaster

# se selecciona el pronosticador
forecaster = ForecasterEquivalentDate(
                 offset    = pd.DateOffset(days=1),
                 n_offsets = 1
             )

In [ ]:
metric, predictions = backtesting_forecaster(
                          forecaster         = forecaster,
                          y                  = data['demanda'],
                          steps              = 24,
                          metric             = 'mean_squared_error',
                          initial_train_size = len(data.loc[:end_train]),
                          refit              = False,
                          n_jobs             = 'auto',
                          verbose            = False,
                          show_progress      = True
                      )

print(f"MSE: {metric:.2f}")

  0%|          | 0/32 [00:00<?, ?it/s]

MSE: 3119371.47


In [ ]:
# grafico lado a lado pronostico y valor real
diferencia = data_test["demanda"] - predictions["pred"]

fig = go.Figure()
trace1 = go.Scatter(x=data_test.index, y=data_test['demanda']/1000, name="Validación", mode="lines", line=dict(color='red'))
trace2 = go.Scatter(x=predictions.index, y=predictions['pred']/1000, name="Predicción", mode="lines", line=dict(color='blue'))
trace3 = go.Scatter(x=predictions.index, y=diferencia/1000, name="Diferencia", mode="lines", line=dict(color='gray'))
fig.add_trace(trace1)
fig.add_trace(trace2)
fig.add_trace(trace3)
fig.update_layout(
    width=600, height=400,
    xaxis_title="Fecha",
    yaxis_title="Demanda [MW]",
    template="simple_white",
)
fig.write_image("prediccion_ingenua.png",  engine="kaleido", scale=5)
fig.show()

# Fig - Modelo polinomio grado 2 de la temperatura


In [ ]:
# codigo ready to use para un modelo polinomial de la temperatura
# importing packages and modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score
import scipy.stats as stats

dataset = data
# degree 2 polynomial fit or quadratic fit
model_temp = np.poly1d(np.polyfit(dataset['temp'],
							dataset['demanda'], 2))

meandem = data['demanda'].mean()
data['temp_original'] = data['temp']
mean = data['temp_original'].mean()
data['temp1'] = model_temp(data['temp_original'])

fig = go.Figure()
trace1 = go.Scatter(x=data['temp_original'], y=data['demanda']/1000, name="Mediciones", mode="markers",
    marker=dict(color='LightSkyBlue', size=0.1, line=dict(color='Blue', width=2)))

model_x = [k for k in range(5, 35)]
model = pd.DataFrame()
model['model_y'] = model_temp(model_x)

trace2 = go.Scatter(x=model_x, y=model['model_y']/1000, name="Ajuste", mode="lines", line=dict(color='red'))

fig.add_trace(trace1)
fig.add_trace(trace2)
fig.add_vrect(x0=mean, x1=mean)

fig.update_layout(
    width=600, height=400,
    xaxis_title="Temperatura [°C]",
    yaxis_title="Demanda [MW]",
    template="simple_white",
)
fig.write_image("demanda_vs_temperatura.png",  engine="kaleido", scale=5)
fig.show()

# Fig - Modelo entrenado sin Exógenas


In [ ]:
end_train = '2023-12-31 23:59:00'
data_train = data.loc[:end_train, :].copy()
data_test = data.loc[end_train:, :].copy()

forecaster = ForecasterAutoreg(
    regressor=LinearRegression(),   # que modelo se usa
    lags=24*7 ,                     # cuanto mira hacia atras
    #weight_func=weight_entropia_ponderada    # funcion de peso

)

metric, predictions = backtesting_forecaster(
                          forecaster         = forecaster,
                          y                  = data['demanda'],
                          #exog               = data[['temp']],
                          # Extra: ,'rango_horario' EMPEORA...
                          steps              = 24,
                          metric             = 'mean_squared_error',
                          initial_train_size = len(data.loc[:end_train]),
                          refit              = False,
                          n_jobs             = 'auto',
                          verbose            = False,
                          show_progress      = True
                      )

print(f"MSE: {metric:.2f}")

diferencia = data_test['demanda']-predictions['pred']
df = pd.DataFrame(diferencia)
df.columns = ["dato"]
df["semana"] = df.index.isocalendar().week

error = []
for i in range(4):
    valor = df[df["semana"] == i+1]["dato"].mean()
    error.append(valor)



  0%|          | 0/32 [00:00<?, ?it/s]

MSE: 1530491.74


In [ ]:
# grafico lado a lado pronostico y valor real
fig = go.Figure()
trace1 = go.Scatter(x=data_test.index, y=data_test['demanda']/1000, name="Validación", mode="lines", line=dict(color='red'))
trace2 = go.Scatter(x=predictions.index, y=predictions['pred']/1000, name="Predicción", mode="lines", line=dict(color='blue'))
trace3 = go.Scatter(x=predictions.index, y=diferencia/1000, name="Diferencia", mode="lines", line=dict(color='gray'))
fig.add_trace(trace1)
fig.add_trace(trace2)
fig.add_trace(trace3)
fig.update_layout(
    width=600, height=400,
    xaxis_title="Fecha",
    yaxis_title="Demanda [MW]",
    template="simple_white",
)
fig.write_image("prediccion_sola.png",  engine="kaleido", scale=5)
fig.show()

# Fig - Modelo entrenado con entropía


In [ ]:
end_train = '2023-12-31 23:59:00'
data_train = data.loc[:end_train, :].copy()
data_test = data.loc[end_train:, :].copy()

meandem = data['demanda'].mean()
data['temp_original'] = data['temp']
mean = data['temp_original'].mean()
data['temp1'] = model_temp(data['temp_original'])


forecaster = ForecasterAutoreg(
    regressor=LinearRegression(),   # que modelo se usa
    lags=24*7 ,                     # cuanto mira hacia atras
    #weight_func=weight_entropia_ponderada    # funcion de peso

)

# Calculo la entropía
entropia_ponderada = [0] * 24*7

for i in range(24*7, len(data.demanda), 1):
    h = ordpy.weighted_permutation_entropy(data.demanda[i-24*7:i], dx=3,taux=1)
    entropia_ponderada.append(h)

data['entropia_ponderada'] = entropia_ponderada

metric, predictions = backtesting_forecaster(
                          forecaster         = forecaster,
                          y                  = data['demanda'],
                          exog               = data[['temp1','no_habil']],
                          steps              = 24,
                          metric             = 'mean_squared_error',
                          initial_train_size = len(data.loc[:end_train]),
                          refit              = False,
                          n_jobs             = 'auto',
                          verbose            = False,
                          show_progress      = True
                      )

print(f"MSE: {metric:.2f}")

diferencia = data_test['demanda']-predictions['pred']
df = pd.DataFrame(diferencia)
df.columns = ["dato"]
df["semana"] = df.index.isocalendar().week

error = []
for i in range(4):
    valor = df[df["semana"] == i+1]["dato"].mean()
    error.append(valor)



  0%|          | 0/32 [00:00<?, ?it/s]

MSE: 1244138.61


In [ ]:
# grafico lado a lado pronostico y valor real
fig = go.Figure()
trace1 = go.Scatter(x=data_test.index, y=data_test['demanda']/1000, name="Validación", mode="lines", line=dict(color='red'))
trace2 = go.Scatter(x=predictions.index, y=predictions['pred']/1000, name="Predicción", mode="lines", line=dict(color='blue'))
trace3 = go.Scatter(x=predictions.index, y=diferencia/1000, name="Diferencia", mode="lines", line=dict(color='gray'))
fig.add_trace(trace1)
fig.add_trace(trace2)
fig.add_trace(trace3)
fig.update_layout(
    width=600, height=400,
    xaxis_title="Fecha",
    yaxis_title="Demanda [MW]",
    template="simple_white",
)
fig.write_image("prediccion_con_temperatura_dia_habil.png",  engine="kaleido", scale=5)
fig.show()